In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 

import sklearn
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV, StratifiedKFold, KFold
from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier 
from sklearn.metrics import confusion_matrix,plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import Perceptron, SGDClassifier, LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn import pipeline
import lightgbm as lgb

## for visualization
import plotly.express as px
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import time 

plt.style.use('ggplot')

### Notebook un peu plus clean (avec les déclaration des paramètres et variables au début) avec randomized search sur du xgb boost et/ou random forest sans pca 

In [ ]:
# parameters 
#xgb
params = {
        'min_child_weight': [1, 3, 5, 7, 10],
        'gamma': [0, 0.1, 0.3, 0.5, 0.7, 1],
        'subsample': [0, 0.5, 1.0],
        'colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1.0],
        'max_depth': [2, 5, 7, 10, 15, 25, 50,100],
        'eta': [0.05, 0.1, 0.3, 0.5,1],
        'n_estimators': [100, 200, 500, 800, 1000, 1200, 1500, 2000]
         }
#rf
space = {
       'n_estimators': [50, 100, 200, 400, 500, 800, 1000, 1200, 1500, 2000],
       'max_features': ['auto', 'sqrt'],
       'max_depth': [2,3,5,7, 10, 15, 25, 50, 70, 100, 150,200],
       'min_samples_split': [1, 2, 5, 10,20],
       'min_samples_leaf': [1, 2, 5, 10, 20],
       'bootstrap': [True,False]
        }
seed = 42
np.random.seed(seed)
kfolds = KFold(n_splits=5, shuffle=True, random_state=seed)

models = {'xgb' : XGBClassifier(objective='binary:logistic'), 'rf': RandomForestClassifier()}
model = 'xgb'

In [ ]:
train = pd.read_csv("../input/bioresponse/train.csv")
test = pd.read_csv("../input/bioresponse/test.csv")
benchmark= pd.read_csv("../input/bioresponse/svm_benchmark.csv")

In [ ]:
Y = train.Activity
X = train.drop('Activity', axis=1)

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
test = scaler.transform(test)

X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.20, random_state=seed)


In [ ]:
start = time.time()

pca = PCA(n_components=1000)
pca.fit(X_train)
X_train, X_test = pca.transform(X_train), pca.transform(X_test)
test = pca.transform(test)
end = time.time()
print(end-start)

In [ ]:
start = time.time()

if model=='rf':
    random_search = RandomizedSearchCV(estimator = models[model], param_distributions = space, scoring='neg_log_loss',
                               n_iter = 100, cv = 10, verbose=2, random_state=seed, n_jobs = -1)
    random_search.fit(X_train,y_train)
    
elif model=='xgb':
    random_search = RandomizedSearchCV(estimator=models[model], param_distributions=params, verbose=2,  n_jobs=4,
                                   scoring = 'neg_log_loss', cv=10, random_state=seed)
    random_search.fit(X_train,y_train)
else:
    print("no model has been defined")
    

end = time.time()
print(end-start)

In [ ]:
start = time.time()

model = random_search.best_estimator_
model.fit(X,Y)
sub = model.predict_proba(test)

end = time.time()
print(end-start)

In [ ]:
submission = pd.DataFrame({'MoleculeId': range(1,test.shape[0]+1),
                        'PredictedProbability': sub[:,1]})
submission.to_csv('submission.csv', index=False)

In [ ]:
## on re train le "meilleur model" sur tout le train set et on fait notre prédiction sur le test set en entier 
#train = pd.read_csv("../input/bioresponse/train.csv")
#test = pd.read_csv("../input/bioresponse/test.csv")
#Y = train.Activity
#X = train.drop('Activity', axis=1)
#scaler = StandardScaler()
##scaler.fit(X)
#X = scaler.transform(X)
#test = scaler.transform(test)

#model.fit(X,Y)
#clf.fit(X,Y)
#sub = clf.predict_proba(test)